In [1]:
import csv
import gc
import json
import pickle
import os

import pandas as pd
import numpy as np
import torch as tr
import torch.nn as nn
import torch.nn.functional as F

## Откроем датасет https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package

In [2]:
with open('datasets/weatherAUS.csv') as f:
    reader = csv.reader(f)
    _data = list(reader)
    df = pd.DataFrame(_data[1:], columns=_data[0])

In [3]:
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NA,NA,W,44,W,...,71,22,1007.7,1007.1,8,NA,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0,NA,NA,WNW,44,NNW,...,44,25,1010.6,1007.8,NA,NA,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0,NA,NA,WSW,46,W,...,38,30,1007.6,1008.7,NA,2,21,23.2,No,No
3,2008-12-04,Albury,9.2,28,0,NA,NA,NE,24,SE,...,45,16,1017.6,1012.8,NA,NA,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1,NA,NA,W,41,ENE,...,82,33,1010.8,1006,7,8,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0,NA,NA,E,31,SE,...,51,24,1024.6,1020.3,NA,NA,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.3,0,NA,NA,NNW,22,SE,...,56,21,1023.5,1019.1,NA,NA,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.9,0,NA,NA,N,37,SE,...,53,24,1021,1016.8,NA,NA,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27,0,NA,NA,SE,28,SSE,...,51,24,1019.4,1016.5,3,2,15.1,26,No,No


In [4]:
model = nn.Sequential(
    nn.Linear(8, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 1),
    nn.Sigmoid()
)

loss_fn = nn.BCELoss()
optimizer = tr.optim.Adam(model.parameters(), lr=1e-3)
print(model)

Sequential(
  (0): Linear(in_features=8, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=1, bias=True)
  (5): Sigmoid()
)


In [5]:
# Берем только числовые колонки
columns = ['RainToday', 'MinTemp', 'MaxTemp', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm']
# Убираем пустые значения
df = df[df[columns]!='NA'].dropna(subset=columns)
# Делим на X, Y
X, Y = df.loc[:, df.columns!='RainToday'], df.loc[:, df.columns=='RainToday']
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,NaN,NaN,13.4,22.9,NaN,NaN,NaN,NaN,NaN,NaN,...,71,22,1007.7,1007.1,NaN,NaN,16.9,21.8,No,NaN
1,NaN,NaN,7.4,25.1,NaN,NaN,NaN,NaN,NaN,NaN,...,44,25,1010.6,1007.8,NaN,NaN,17.2,24.3,No,NaN
2,NaN,NaN,12.9,25.7,NaN,NaN,NaN,NaN,NaN,NaN,...,38,30,1007.6,1008.7,NaN,NaN,21,23.2,No,NaN
3,NaN,NaN,9.2,28,NaN,NaN,NaN,NaN,NaN,NaN,...,45,16,1017.6,1012.8,NaN,NaN,18.1,26.5,No,NaN
4,NaN,NaN,17.5,32.3,NaN,NaN,NaN,NaN,NaN,NaN,...,82,33,1010.8,1006,NaN,NaN,17.8,29.7,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145454,NaN,NaN,3.5,21.8,NaN,NaN,NaN,NaN,NaN,NaN,...,59,27,1024.7,1021.2,NaN,NaN,9.4,20.9,No,NaN
145455,NaN,NaN,2.8,23.4,NaN,NaN,NaN,NaN,NaN,NaN,...,51,24,1024.6,1020.3,NaN,NaN,10.1,22.4,No,NaN
145456,NaN,NaN,3.6,25.3,NaN,NaN,NaN,NaN,NaN,NaN,...,56,21,1023.5,1019.1,NaN,NaN,10.9,24.5,No,NaN
145457,NaN,NaN,5.4,26.9,NaN,NaN,NaN,NaN,NaN,NaN,...,53,24,1021,1016.8,NaN,NaN,12.5,26.1,No,NaN


In [6]:
# Приготовим данные, приведем X в float64, а Y в int64
X[columns[1:]] = X[columns[1:]].astype({col: float for col in columns[1:]})
Y = Y.RainToday.map(dict(Yes=1, No=0))

/var/folders/bg/6fjjsrn55s3g6s3781w40rtc0000gq/T/ipykernel_84489/3023816294.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[columns[1:]] = X[columns[1:]].astype({col: float for col in columns[1:]})


In [7]:
X.shape

(125347, 22)

In [8]:
X_tensor = tr.tensor(X[columns[1:]].values)
Y_tensor = tr.tensor(Y.values)

model.train()
for batch, (_x, _y) in enumerate(zip(X_tensor, Y_tensor)):
    _x, _y = _x.to('cpu'), _y.to('cpu').float()

    # Compute prediction error
    pred = model(_x.float())
    loss = loss_fn(pred[0], _y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 1000 == 0:
        loss, current = loss.item(), batch + 1
        print(f"loss: {loss:>7f}  [{current:>5d}/{X_tensor.shape[0]:>5d}]")

loss: 4.983236  [    1/125347]
loss: 0.000000  [ 1001/125347]
loss: 0.000000  [ 2001/125347]
loss: 0.000000  [ 3001/125347]
loss: 0.000000  [ 4001/125347]
loss: 0.000000  [ 5001/125347]
loss: 0.000000  [ 6001/125347]
loss: 0.000000  [ 7001/125347]
loss: 100.000000  [ 8001/125347]
loss: 0.000000  [ 9001/125347]
loss: 100.000000  [10001/125347]
loss: 0.000000  [11001/125347]
loss: 0.000000  [12001/125347]
loss: 0.000000  [13001/125347]
loss: 100.000000  [14001/125347]
loss: 0.000000  [15001/125347]
loss: 0.000000  [16001/125347]
loss: 0.000000  [17001/125347]
loss: 0.000000  [18001/125347]
loss: 0.000000  [19001/125347]
loss: 0.000000  [20001/125347]
loss: 100.000000  [21001/125347]
loss: 0.000000  [22001/125347]
loss: 0.000000  [23001/125347]
loss: 0.000000  [24001/125347]
loss: 0.000000  [25001/125347]
loss: 0.000000  [26001/125347]
loss: 100.000000  [27001/125347]
loss: 100.000000  [28001/125347]
loss: 0.000000  [29001/125347]
loss: 100.000000  [30001/125347]
loss: 0.000000  [31001/12

In [9]:
model(X_tensor.to('cpu').float())

tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], grad_fn=<SigmoidBackward0>)

In [10]:
model.__call__(X_tensor.to('cpu').float())

tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], grad_fn=<SigmoidBackward0>)

In [11]:
model(tr.tensor(np.array([[1., 2., 3., 4., 5., 6., 7., 8.]])).to('cpu').float())

tensor([[0.4163]], grad_fn=<SigmoidBackward0>)

## Сохраняем модель

In [13]:
tr.save(model, 'models/pytorch-binary_cls_model.pth')
# model = torch.load('models/pytorch-binary_cls_model.pth')
# model.eval()


model_scripted = tr.jit.script(model) # Export to TorchScript
model_scripted.save('models/pytorch-binary_cls_model-jit.pth') # Save
# model = torch.jit.load('models/pytorch-binary_cls_model-scripted.pth')
# model.eval()
with open('models/pytorch-binary_cls_model.pckl', 'wb') as f:
    pickle.dump(model, f)

In [50]:
tr.onnx.export(
    model,                                                   # model being run
    tr.tensor(np.array([[1., 2., 3., 4., 5., 6., 7., 8.]], dtype=np.float32)),  # model input (or a tuple for multiple inputs)
    "models/pytorch-binary_cls_model.onnx",  # where to save the model (can be a file or file-like object)
    export_params=True,        # store the trained parameter weights inside the model file
    opset_version=10,          # the ONNX version to export the model to
    do_constant_folding=True,  # whether to execute constant folding for optimization
    input_names = ['input'],   # the model's input names
    output_names = ['output'], # the model's output names
)

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [56]:
import onnxruntime as ort

ort_sess = ort.InferenceSession("models/pytorch-binary_cls_model.onnx")
outputs = ort_sess.run(None, {'input': np.array([[1., 2., 3., 4., 5., 6., 7., 8.]], dtype=np.float32)})
print(outputs)

[array([[0.41630384]], dtype=float32)]


## Columns
* 'MinTemp'
* 'MaxTemp'
* 'Humidity9am'
* 'Humidity3pm'
* 'Pressure9am'
* 'Pressure3pm'
* 'Temp9am'
* 'Temp3pm'

Принимает на вход массив из 8 float64 числа. Выдает тензор с числом

## Requirements
* torch
* torchvision
* pandas